# Developing Text Classification Model for Fake News Detection

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchtext.data.functional import to_map_style_dataset

import en_core_web_sm
import re
import string
import time

import pandas as pd

In [62]:
nlp = en_core_web_sm.load()

random_state = 42

DATA_FILE = '/data/politifact_factcheck_data.json'
data = pd.read_json(DATA_FILE, lines=True)

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Binarize the labels
binary_map = {
    'true': 1,
    'mostly-true': 1, 
    'half-true': 1,
    'mostly-false': 0,
    'false': 0,
    'pants-fire': 0 
}

data['binary_verdict'] = data['verdict'].map(binary_map)

In [5]:
# Clean the dataset
def clean_text(text):
    '''Make text lowercase, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('.*?¿', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub(r" +", " ", text)
    return text

def remove_stop_words(text):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop]
    return " ".join(token for token in tokens)

def remove_spaces(text):
    text = re.sub(r" +", " ", text)
    return text

data['cleaned_statement'] = data['statement'].apply(clean_text)
data['cleaned_statement'] = data['cleaned_statement'].apply(remove_stop_words)
data['cleaned_statement'] = data['cleaned_statement'].apply(remove_spaces)

In [6]:
class PolitifactDataset(Dataset):
    
    def __init__(self, text, labels):
        self.labels = labels.values
        self.text = text.values
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        label = self.labels[idx]
        text = self.text[idx]
        sample = (label, text)
        return sample

In [7]:
dataset = PolitifactDataset(data["cleaned_statement"],
                            data["binary_verdict"])

In [8]:
print('\nFirst element of dataset: ', next(iter(dataset)), '\n')
print('Length of dataset: ', len(dataset), '\n')


First element of dataset:  (1, 'john mccain opposed bankruptcy protections families bankruptcy medical expenses nt pay') 

Length of dataset:  21152 



In [9]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [10]:
tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)
        
vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [11]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) 

In [12]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [47]:
import torch.nn as nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class, hidden_dim=128, dropout_rate=0.5):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = self.dropout(embedded)
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x)



In [48]:
num_class = len(set([label for (label, text) in dataset]))
vocab_size = len(vocab)
emsize = 128
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [49]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [50]:
train_dataset, test_dataset = random_split(dataset, 
                                           [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))], 
                                           generator=torch.Generator().manual_seed(random_state))

train_dataset = to_map_style_dataset(train_dataset)
test_dataset = to_map_style_dataset(test_dataset)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=8, shuffle=False, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=8, shuffle=False, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [51]:
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

In [52]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val
                                   ))
    print('-' * 59)

| epoch   1 |   500/ 2010 batches | accuracy    0.531
| epoch   1 |  1000/ 2010 batches | accuracy    0.562
| epoch   1 |  1500/ 2010 batches | accuracy    0.591
| epoch   1 |  2000/ 2010 batches | accuracy    0.605
-----------------------------------------------------------
| end of epoch   1 | time:  4.10s | valid accuracy    0.551 
-----------------------------------------------------------
| epoch   2 |   500/ 2010 batches | accuracy    0.604
| epoch   2 |  1000/ 2010 batches | accuracy    0.623
| epoch   2 |  1500/ 2010 batches | accuracy    0.636
| epoch   2 |  2000/ 2010 batches | accuracy    0.636
-----------------------------------------------------------
| end of epoch   2 | time:  4.15s | valid accuracy    0.583 
-----------------------------------------------------------
| epoch   3 |   500/ 2010 batches | accuracy    0.644
| epoch   3 |  1000/ 2010 batches | accuracy    0.661
| epoch   3 |  1500/ 2010 batches | accuracy    0.658
| epoch   3 |  2000/ 2010 batches | accuracy

In [53]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('Test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
Test accuracy    0.663


In [60]:
import os
import torch

# Create the directory if it doesn't exist
save_dir = 'model/'
os.makedirs(save_dir, exist_ok=True)

# Specify the file path where you want to save the model
model_path = os.path.join(save_dir, 'TextClassificationModel.pth')

# Save the model
torch.save(model.state_dict(), model_path)

print(f'Model saved to {model_path}')


Model saved to kaggle/working/TextClassificationModel.pth


In [ ]:
import pickle
output = open('vocab/vocab.pth', 'wb')
pickle.dump(vocab, output)
output.close()